In [4]:
import math
import numpy as np
from collections import Counter
import csv
# Note: please don't add any new package, you should solve this problem using only the packages above.
# However, importing the Python standard library is allowed: https://docs.python.org/3/library/
#-------------------------------------------------------------------------
'''
    Part 1: Decision Tree (with Discrete Attributes) -- 60 points --
    In this problem, you will implement the decision tree method for classification problems.
    You could test the correctness of your code by typing `pytest -v test1.py` in the terminal.
'''

#-----------------------------------------------
class Node:
    '''
        Decision Tree Node (with discrete attributes)
        Inputs: 
            X: the data instances in the node, a numpy matrix of shape p by n.
               Each element can be int/float/string.
               Here n is the number data instances in the node, p is the number of attributes.
            Y: the class labels, a numpy array of length n.
               Each element can be int/float/string.
            i: the index of the attribute being tested in the node, an integer scalar 
            C: the dictionary of attribute values and children nodes. 
               Each (key, value) pair represents an attribute value and its corresponding child node.
            isleaf: whether or not this node is a leaf node, a boolean scalar
            p: the label to be predicted on the node (i.e., most common label in the node).
    '''
    def __init__(self,X,Y, i=None,C=None, isleaf= False,p=None):
        self.X = X
        self.Y = Y
        self.i = i
        self.C= C
        self.isleaf = isleaf
        self.p = p

#-----------------------------------------------
class Tree(object):
    '''
        Decision Tree (with discrete attributes). 
        We are using ID3(Iterative Dichotomiser 3) algorithm. So this decision tree is also called ID3.
    '''
    #--------------------------
    @staticmethod
    def entropy(Y):
        '''
            Compute the entropy of a list of values.
            Input:
                Y: a list of values, a numpy array of int/float/string values.
            Output:
                e: the entropy of the list of values, a float scalar
            Hint: you could use collections.Counter.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
             
        counter = Counter(Y)  #Count the number of each label. For example: yes: 5, no: 3
        total = len(Y)     #Count total number of labels in the dataset. For example: 8
        e = 0
        for count in counter.values():
            p = count/total #probability
            e -= p * math.log2(p) #entropy

        #########################################
        return e 
    
    
            
    #--------------------------
    @staticmethod
    def conditional_entropy(Y,X):
        '''
            Compute the conditional entropy of y given x. The conditional entropy H(Y|X) means average entropy of children nodes, given attribute X. Refer to https://en.wikipedia.org/wiki/Information_gain_in_decision_trees
            Input:
                X: a list of values , a numpy array of int/float/string values. The size of the array means the number of instances/examples. X contains each instance's attribute value. 
                Y: a list of values, a numpy array of int/float/string values. Y contains each instance's corresponding target label. For example X[0]'s target label is Y[0]
            Output:
                ce: the conditional entropy of y given x, a float scalar
        '''
        #########################################
        ## INSERT YOUR CODE HERE

        
        unique_values = np.unique(X)  #Get all the unique attributes. For example: 
        total = len(Y)
        ce = 0
        for val in unique_values:
            subset_Y = Y[X == val]
            p = len(subset_Y)/total
            subset_Y_entropy = Tree.entropy(subset_Y)
            ce += p * subset_Y_entropy
        
 
        #########################################
        return ce 
    
    
    
    #--------------------------
    @staticmethod
    def information_gain(Y,X):
        '''
            Compute the information gain of y after spliting over attribute x
            InfoGain(Y,X) = H(Y) - H(Y|X) 
            Input:
                X: a list of values, a numpy array of int/float/string values.
                Y: a list of values, a numpy array of int/float/string values.
            Output:
                g: the information gain of y after spliting over x, a float scalar
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        ## IG = parent entropy - average childnode entropy
        
        H_Y = Tree.entropy(Y)
        H_Y_given_X = Tree.conditional_entropy(Y, X)
        g = H_Y - H_Y_given_X
 
        #########################################
        return g


    #--------------------------
    @staticmethod
    def best_attribute(X,Y):
        '''
            Find the best attribute to split the node. 
            Here we use information gain to evaluate the attributes. 
            If there is a tie in the best attributes, select the one with the smallest index.
            Input:
                X: the feature matrix, a numpy matrix of shape p by n. 
                   Each element can be int/float/string.
                   Here n is the number data instances in the node, p is the number of attributes.
                Y: the class labels, a numpy array of length n. Each element can be int/float/string.
            Output:
                i: the index of the attribute to split, an integer scalar
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        
        num_attributes = X.shape[0]
        best_gain = -1
        i = -1
        
        for j in range(num_attributes):
            gain = Tree.information_gain(Y, X[j])
            if gain > best_gain: 
                best_gain = gain
                i = j
            elif gain == best_gain and j < i: 
                i = j 
 
        #########################################
        return i

        
    #--------------------------
    @staticmethod
    def split(X,Y,i):
        '''
            Split the node based upon the i-th attribute.
            (1) split the matrix X based upon the values in i-th attribute
            (2) split the labels Y based upon the values in i-th attribute
            (3) build children nodes by assigning a submatrix of X and Y to each node
            (4) build the dictionary to combine each  value in the i-th attribute with a child node.
    
            Input:
                X: the feature matrix, a numpy matrix of shape p by n.
                   Each element can be int/float/string.
                   Here n is the number data instances in the node, p is the number of attributes.
                Y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
                i: the index of the attribute to split, an integer scalar
            Output:
                C: the dictionary of attribute values and children nodes. 
                   Each (key, value) pair represents an attribute value and its corresponding child node.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        C = {}
        
        
        unique_vals = np.unique(X[i, :])

        for value in unique_vals:
            indices = np.where(X[i] == value)

            X_sub = X[:, indices[0]]
            Y_sub = Y[indices]

            child_node = Node(X_sub, Y_sub)

            C[value] = child_node


        #########################################
        return C

    #--------------------------
    @staticmethod
    def stop1(Y):
        '''
            Test condition 1 (stop splitting): whether or not all the instances have the same label. 
    
            Input:
                Y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
            Output:
                s: whether or not Condition 1 holds, a boolean scalar. 
                True if all labels are the same. Otherwise, false.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        
        s = np.all(Y == Y[0])       
        
        #########################################
        return s
    
    #--------------------------
    @staticmethod
    def stop2(X):
        '''
            Test condition 2 (stop splitting): whether or not all the instances have the same attribute values. 
            Input:
                X: the feature matrix, a numpy matrix of shape p by n.
                   Each element can be int/float/string.
                   Here n is the number data instances in the node, p is the number of attributes.
            Output:
                s: whether or not Conidtion 2 holds, a boolean scalar. 
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        
        
        
        if np.all(X == X[:, [0]], axis=1).all():
            s = True
        else:
            s = False
    
        #########################################
        return s
    
            
    #--------------------------
    @staticmethod
    def most_common(Y):
        '''
            Get the most-common label from the list Y. 
            Input:
                Y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
                   Here n is the number data instances in the node.
            Output:
                y: the most common label, a scalar, can be int/float/string.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        Y = np.array(Y)
        
        counter = Counter(Y)
        y = counter.most_common(1)[0][0]
 
        #########################################
        return y
    
    
    
    #--------------------------
    @staticmethod
    def build_tree(t):
        '''
            Recursively build tree nodes.
            Input:
                t: a node of the decision tree, without the subtree built.
                t.X: the feature matrix, a numpy float matrix of shape p by n.
                   Each element can be int/float/string.
                    Here n is the number data instances, p is the number of attributes.
                t.Y: the class labels of the instances in the node, a numpy array of length n.
                t.C: the dictionary of attribute values and children nodes. 
                   Each (key, value) pair represents an attribute value and its corresponding child node.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        t.p = Tree.most_common(t.Y)
        if Tree.stop1(t.Y) == False and Tree.stop2(t.X) == False:
            t.i = Tree.best_attribute(t.X, t.Y)
            t.C = Tree.split(t.X, t.Y, t.i) 

            for val in t.C.values():
                Tree.build_tree(val)
        else:
            t.isleaf=True


        
        
 
        #########################################
    
    
    #--------------------------
    @staticmethod
    def train(X, Y):
        '''
            Given a training set, train a decision tree. 
            Input:
                X: the feature matrix, a numpy matrix of shape p by n.
                   Each element can be int/float/string.
                   Here n is the number data instances in the training set, p is the number of attributes.
                Y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
            Output:
                t: the root of the tree.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        
        
        t = Node(X, Y)
        Tree.build_tree(t)
 
        #########################################
        return t
    
    
    
    #--------------------------
    @staticmethod
    def inference(t,x):
        '''
            Given a decision tree and one data instance, infer the label of the instance recursively. 
            Input:
                t: the root of the tree.
                x: the attribute vector, a numpy vectr of shape p.
                   Each attribute value can be int/float/string.
            Output:
                y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
        '''
        #########################################
        ## INSERT YOUR CODE HERE

        if t.isleaf:
            y = t.p
            return y


        attribute_value = x[t.i]
        if attribute_value in t.C:
            return Tree.inference(t.C[attribute_value], x)
        else:
            y = t.p

 
        #########################################
        return y
    
    #--------------------------
    @staticmethod
    def predict(t,X):
        '''
            Given a decision tree and a dataset, predict the labels on the dataset. 
            Input:
                t: the root of the tree.
                X: the feature matrix, a numpy matrix of shape p by n.
                   Each element can be int/float/string.
                   Here n is the number data instances in the dataset, p is the number of attributes.
            Output:
                Y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
        '''
        #########################################
        ## INSERT YOUR CODE HERE

               
        Y = np.array([Tree.inference(t, X[:, i]) for i in range(X.shape[1])])

        #########################################
        return Y



    #--------------------------
    @staticmethod
    def load_dataset(filename = 'data1.csv'):
        '''
            Load dataset 1 from the CSV file: 'data1.csv'. 
            The first row of the file is the header (including the names of the attributes)
            In the remaining rows, each row represents one data instance.
            The first column of the file is the label to be predicted.
            In remaining columns, each column represents an attribute.
            Input:
                filename: the filename of the dataset, a string.
            Output:
                X: the feature matrix, a numpy matrix of shape p by n.
                   Each element can be int/float/string.
                   Here n is the number data instances in the dataset, p is the number of attributes.
                Y: the class labels, a numpy array of length n.
                   Each element can be int/float/string.
        '''
        #########################################
        ## INSERT YOUR CODE HERE
        with open(filename, 'r') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader)
            data = list(reader)
        data = np.array(data)

        X = data[:, 1:].T
        Y = data[:, 0]
 
        #########################################
        return X,Y


In [5]:
def accuracy(Y_pred, Y_true):
    correct = np.sum(Y_pred == Y_true)
    total = len(Y_true)
    acc = correct / total
    return acc


X, Y = Tree.load_dataset('data1.csv')

# Train and test split
Xtrain, Ytrain = X[:, ::2], Y[::2]
Xtest, Ytest = X[:, 1::2], Y[1::2]
# Building the decision tree
tree = Tree.train(Xtrain, Ytrain)
# Predicting on the training and test sets
Ytrain_pred = Tree.predict(tree, Xtrain)
Ytest_pred = Tree.predict(tree, Xtest)
# Evaluate the performance
train_accuracy = accuracy(Ytrain_pred, Ytrain)
test_accuracy = accuracy(Ytest_pred, Ytest)

print(f"Training Accuracy: {train_accuracy * 100}%")
print(f"Test Accuracy: {test_accuracy * 100}%")

Training Accuracy: 100.0%
Test Accuracy: 71.42857142857143%


In [21]:
        ## Testing...
Weather = ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 
           'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 
           'Sunny', 'Overcast', 'Overcast', 'Rain'] 

Temperature = ['High', 'High', 'High', 'Low', 'Low', 
           'High', 'High', 'High', 'Low', 'Medium', 
           'Low', 'Medium', 'Medium', 'Medium'] 

Random_feature = [0,0,9,9,9,0,9,0,9,9,9,9,9,0]

Play_Tennis = ['No', 'No', 'Yes', 'Yes', 'Yes', 
               'No', 'Yes', 'No', 'Yes', 'Yes',  
               'Yes', 'Yes', 'Yes', 'No']

ig_weather = Tree.information_gain(Play_Tennis, Weather)
print(f'Information Gain if we split Weather: {ig_weather}')

ig_temp = Tree.information_gain(Play_Tennis, Temperature)
print(f'Information Gain if we split Temperature: {ig_temp}')

ig_rf = Tree.information_gain(Play_Tennis, Random_feature)
print(f'Information Gain if we split Random Feature: {ig_rf}')



features = np.array([['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
              ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
              ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High']])

target = np.array(['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No'])

best_attr = Tree.best_attribute(features,target)
print(f'Best attribute to split on: {best_attr}')

split_test = Tree.split(features, target, best_attr)
for val, node in split_test.items():
    print(f'Attribute: {val}, Nodes: X={node.X}, Y={node.Y}')

Information Gain if we split Weather: 0.24674981977443933
Information Gain if we split Temperature: 0.31493685137324057
Information Gain if we split Random Feature: 0.9402859586706311
Best attribute to split on: 0
Attribute: Overcast, Nodes: X=[['Overcast' 'Overcast' 'Overcast' 'Overcast']
 ['Hot' 'Cool' 'Mild' 'Hot']
 ['High' 'Normal' 'High' 'Normal']], Y=['Yes' 'Yes' 'Yes' 'Yes']
Attribute: Rain, Nodes: X=[['Rain' 'Rain' 'Rain' 'Rain' 'Rain']
 ['Mild' 'Cool' 'Cool' 'Mild' 'Mild']
 ['High' 'Normal' 'Normal' 'Normal' 'High']], Y=['Yes' 'Yes' 'No' 'Yes' 'No']
Attribute: Sunny, Nodes: X=[['Sunny' 'Sunny' 'Sunny' 'Sunny' 'Sunny']
 ['Hot' 'Hot' 'Mild' 'Cool' 'Mild']
 ['High' 'High' 'High' 'Normal' 'Normal']], Y=['No' 'No' 'No' 'Yes' 'Yes']


In [22]:
a = Tree.train(features, target)
print(a)